# Exotic Derivatives Trading Desk Analysis

**As of 16 May 2025**, we will:
1. Fetch market data  
2. Instantiate each of our four option classes  
3. Price & (where available) compute delta  
4. Summarize hedges for the combined portfolio  


In [1]:
# Install needed packages (run this cell once)
!pip install --quiet yfinance scipy

In [2]:
# Cell 2: imports and discount-factor setup
import datetime
import numpy as np
import pandas as pd
import yfinance as yf

from derivatives import (
    EuropeanCall,
    AmericanPut,
    UpAndInBarrierCall,
    BasketCall
)

# Flat 4% risk-free curve → discount factor function
r = 0.04
discount = lambda T: np.exp(-r * T)


In [3]:
# Cell 2.5: Yield‐curve bootstrapping

import numpy as np
from scipy.interpolate import interp1d

# 1. Market tenors (years) and continuous zero rates (as decimals)
tenors     = np.array([1/12, 3/12, 6/12, 1.0, 3.0, 5.0, 10.0, 15.0])
zero_rates = np.array([
    0.037994,  # 1 m
    0.037990,  # 3 m
    0.038575,  # 6 m
    0.035670,  # 1 y
    0.035185,  # 3 y
    0.038823,  # 5 y
    0.043145,  # 10 y
    0.045320   # 15 y
])

# 2. Discount factors at those tenors
discount_factors = np.exp(-zero_rates * tenors)

# 3. Build a linear zero‐rate interpolator
zero_interp = interp1d(
    tenors,
    zero_rates,
    kind="linear",
    fill_value="extrapolate",
    assume_sorted=True
)

# 4. Override discount(T) to use the interpolated curve
discount = lambda T: np.exp(-zero_interp(T) * T)

# Quick check
print("DF(0.5 y) =", discount(0.5))
print("DF(2 y)   =", discount(2.0))


DF(0.5 y) = 0.9808973137225787
DF(2 y)   = 0.931596963894417


In [4]:
# Cell 3: Market inputs as of 16 May 2025

trade_date = "2025-05-16"
next_day   = (
    datetime.datetime.strptime(trade_date, "%Y-%m-%d")
    + datetime.timedelta(days=1)
).strftime("%Y-%m-%d")

tickers = ["BHP.AX","CBA.AX","WES.AX","CSL.AX","WDS.AX","MQG.AX"]
df = yf.download(tickers, start=trade_date, end=next_day)["Close"]
S0 = df.loc[trade_date].to_dict()


print("Spot prices:", S0)

print("Flat r:", r)


YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  6 of 6 completed

Spot prices: {'BHP.AX': 39.720001220703125, 'CBA.AX': 169.66000366210938, 'CSL.AX': 241.82000732421875, 'MQG.AX': 207.35000610351562, 'WDS.AX': 21.920000076293945, 'WES.AX': 82.55999755859375}
Flat r: 0.04


In [5]:
# Cell 3.1: Estimate 1-year historical daily vol (annualised)

import numpy as np

# Pull one year of daily closes up to 16 May 2025
hist_start = (
    datetime.datetime.strptime(trade_date, "%Y-%m-%d")
    - datetime.timedelta(days=365)
).strftime("%Y-%m-%d")

hist = yf.download(tickers, start=hist_start, end=trade_date)["Close"]

# Compute daily returns & annualise
rets     = hist.pct_change().dropna()
hist_vol = (rets.std() * np.sqrt(252)).to_dict()

# Use these as your vol inputs
vol = hist_vol

# Quick check printout
print("Using 1y historical vols:")
for t, v in vol.items():
    print(f"  {t}: {v:.2%}")

[*********************100%***********************]  6 of 6 completed

Using 1y historical vols:
  BHP.AX: 22.72%
  CBA.AX: 21.46%
  CSL.AX: 19.19%
  MQG.AX: 24.65%
  WDS.AX: 28.96%
  WES.AX: 20.88%


In [6]:
# Cell 4: Instantiate & price

# 1) BHP European Call
T1         = (datetime.datetime(2027,9,15) - datetime.datetime(2025,5,16)).days/365
bhp_call   = EuropeanCall(
    S0       = S0["BHP.AX"],
    K        = 0.98 * S0["BHP.AX"],
    T        = T1,
    discount = discount,
    sigma    = vol["BHP.AX"]
)
price_bhp, delta_bhp = bhp_call.price(), bhp_call.delta()

# 2) CBA American Put
T2        = (datetime.datetime(2026,5,15) - datetime.datetime(2025,5,16)).days/365
cba_put   = AmericanPut(
    S0       = S0["CBA.AX"],
    K        = 170.0,
    T        = T2,
    discount = discount,
    sigma    = vol["CBA.AX"]
)
price_cba, delta_cba = cba_put.price(), cba_put.delta()

# 3) WES Up-and-In Barrier Call
wes_bar   = UpAndInBarrierCall(
    S0       = S0["WES.AX"],
    K        = 80.0,
    T        = T1,
    discount = discount,
    sigma    = vol["WES.AX"],
    barrier  = 100.0
)
price_wes, delta_wes = wes_bar.price(), wes_bar.delta()

# 4) Four-Stock Basket Call
T3         = (datetime.datetime(2025,7,17) - datetime.datetime(2025,5,16)).days/365
basket     = BasketCall(
    S0_list    = [S0[t] for t in ["BHP.AX","CSL.AX","WDS.AX","MQG.AX"]],
    weights    = [0.10,0.35,0.15,0.40],
    K          = 175.0,
    T          = T3,
    discount   = discount,
    sigma_list = [vol[t] for t in ["BHP.AX","CSL.AX","WDS.AX","MQG.AX"]]
)
price_bask, delta_bask = basket.price(), basket.delta()

# Quick print
print(f"BHP Call     → Price: {price_bhp:.2f}, Δ: {delta_bhp:.4f}")
print(f"CBA Put      → Price: {price_cba:.2f}, Δ: {delta_cba:.4f}")
print(f"WES Barrier  → Price: {price_wes:.2f}, Δ: {delta_wes:.4f}")
print(f"Basket Call  → Price: {price_bask:.2f}, Δ: {delta_bask:.4f}")


BHP Call     → Price: 7.37, Δ: 0.6806
CBA Put      → Price: 169.81, Δ: -0.0011
WES Barrier  → Price: 14.52, Δ: 19.3251
Basket Call  → Price: 0.59, Δ: 0.0811


In [7]:
# Cell 4.5: (Debug) verify time-to-expiry values
print(f"T_BHP (yrs):   {T1:.4f}")
print(f"T_CBA (yrs):   {T2:.4f}")
print(f"T_Basket (yrs):{T3:.4f}")


T_BHP (yrs):   2.3342
T_CBA (yrs):   0.9973
T_Basket (yrs):0.1699


In [8]:
# Cell 5: Build & format DataFrame of Price & Greeks safely

import numpy as np
import pandas as pd

def safe_vega(opt):
    """Call opt.vega(), but return NaN if it fails (e.g. BasketCall)."""
    try:
        return opt.vega()
    except Exception:
        return np.nan

# 1) Build raw Greeks table
df_greeks = pd.DataFrame({
    "Instrument": ["BHP Call", "CBA Put", "WES Barrier", "Basket Call"],
    "Price":      [price_bhp, price_cba, price_wes, price_bask],
    "Delta":      [delta_bhp, delta_cba, delta_wes, delta_bask],
    "Vega": [
    bhp_call.vega(),
    cba_put.vega(),
    wes_bar.vega(),
    basket.vega()     # direct now that it exists
],

    "Theta":      [
        bhp_call.theta(),
        cba_put.theta(),
        wes_bar.theta(),
        basket.theta()
    ]
})

# 2) Round all numeric columns to 2 decimals
df_fmt = df_greeks.copy()
for col in ["Price", "Delta", "Vega", "Theta"]:
    df_fmt[col] = df_fmt[col].round(2)

# 3) Format Price as currency
df_fmt["Price"] = df_fmt["Price"].map("${:,.2f}".format)

# 4) Display
df_fmt


,Instrument,Price,Delta,Vega,Theta
0,BHP Call,$7.37,0.68,21.68,-1.75
1,CBA Put,$169.81,-0.00,6.06,0.00
2,WES Barrier,$14.52,19.33,1199.20,66.31
3,Basket Call,$0.59,0.08,-62.13,0.14


In [9]:
# Cell 6: Compute delta‐hedge quantities
df_fmt["Hedge Qty"] = -df_fmt["Delta"]
df_fmt


,Instrument,Price,Delta,Vega,Theta,Hedge Qty
0,BHP Call,$7.37,0.68,21.68,-1.75,-0.68
1,CBA Put,$169.81,-0.00,6.06,0.00,0.00
2,WES Barrier,$14.52,19.33,1199.20,66.31,-19.33
3,Basket Call,$0.59,0.08,-62.13,0.14,-0.08


In [10]:
# Cell 7: Dynamic narrative summary with automated buy/sell logic
from IPython.display import Markdown, display

# Helper to choose action
def action_str(qty):
    return "buying" if qty > 0 else "selling"

# Instrument metadata: (row_index, display_name, ticker, contract_type)
legs = [
    (0, "BHP European Call",        "BHP", "call"),
    (1, "CBA American Put",         "CBA", "put"),
    (2, "WES Up-and-In Barrier Call","WES","barrier call"),
]

text = "## Portfolio Hedge Summary\n\n"

# Individual instruments
for idx, name, ticker, ctype in legs:
    price = df_fmt.loc[idx, "Price"]
    delta = df_fmt.loc[idx, "Delta"]
    vega  = df_fmt.loc[idx, "Vega"]
    theta = df_fmt.loc[idx, "Theta"]
    hqty  = df_fmt.loc[idx, "Hedge Qty"]
    action = action_str(hqty)

    text += (
        f"- **{name}**  \n"
        f"  Price = **{price}**, Δ = {delta}, Vega = {vega}, Theta = {theta}  \n"
        f"  → Hedge by **{action}** {abs(hqty):.2f} shares of {ticker} per {ctype}.\n\n"
    )

# Basket leg
basket_delta = df_fmt.loc[3, "Delta"]
basket_hq    = df_fmt.loc[3, "Hedge Qty"]
basket_price = df_fmt.loc[3, "Price"]
basket_vega  = df_fmt.loc[3, "Vega"]
basket_theta = df_fmt.loc[3, "Theta"]
action_b    = action_str(basket_hq)

text += (
    f"- **Four-Stock Basket Call**  \n"
    f"  Price = **{basket_price}**, Δ = {basket_delta}, Vega = {basket_vega}, Theta = {basket_theta}  \n"
    f"  → Hedge basket by trading each underlying:  \n"
)

# weights in same order as we built the basket
weights = [0.10, 0.35, 0.15, 0.40]
tickers = ["BHP", "CSL", "WDS", "MQG"]

for w, t in zip(weights, tickers):
    sub_qty   = w * basket_hq
    sub_action= action_str(sub_qty)
    text += f"    - **{sub_action}** {abs(sub_qty):.2f} shares of {t} (weight {w*100:.0f}%)  \n"

display(Markdown(text))


## Portfolio Hedge Summary

- **BHP European Call**  
  Price = **$7.37**, Δ = 0.68, Vega = 21.68, Theta = -1.75  
  → Hedge by **selling** 0.68 shares of BHP per call.

- **CBA American Put**  
  Price = **$169.81**, Δ = -0.0, Vega = 6.06, Theta = 0.0  
  → Hedge by **selling** 0.00 shares of CBA per put.

- **WES Up-and-In Barrier Call**  
  Price = **$14.52**, Δ = 19.33, Vega = 1199.2, Theta = 66.31  
  → Hedge by **selling** 19.33 shares of WES per barrier call.

- **Four-Stock Basket Call**  
  Price = **$0.59**, Δ = 0.08, Vega = -62.13, Theta = 0.14  
  → Hedge basket by trading each underlying:  
    - **selling** 0.01 shares of BHP (weight 10%)  
    - **selling** 0.03 shares of CSL (weight 35%)  
    - **selling** 0.01 shares of WDS (weight 15%)  
    - **selling** 0.03 shares of MQG (weight 40%)  
